* https://www.learnpytorch.io/05_pytorch_going_modular/


# 05. Going Modular

* https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/going_modular/05_pytorch_going_modular_cell_mode.ipynb

## 1. Get data

In [7]:
%%writefile going_modular/going_modular/data_import.py
import os
import requests
import zipfile
from pathlib import Path

def get_data(url, file):
    '''
    Download and extract data

    Args:
        url: url to download data from
        file: name of zip file
    '''
    # Setup path to data folder
    data_path = Path("data/")
    image_path = data_path / file

    # If the image folder doesn't exist, download it and prepare it... 
    if image_path.is_dir():
        print(f"{image_path} directory exists.")
    else:
        print(f"Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

    if not os.path.exists(image_path):
        # Download data
        with open(f"{image_path}.zip", "wb") as f:
            request = requests.get(url)
            print("Downloading data from", url)
            f.write(request.content)

        # Unzip data
        with zipfile.ZipFile(f"{image_path}.zip", "r") as zip_ref:
            print("Unzipping data...") 
            zip_ref.extractall(image_path)

        # Remove zip file
        os.remove(f"{image_path}.zip")

Overwriting going_modular/going_modular/data_import.py


## 2. Create Datasets and DataLoaders

Now we'll turn the image dataset into PyTorch `Dataset`'s and `DataLoader`'s. 

* https://ipython.readthedocs.io/en/stable/interactive/magics.html#cell-magics

We can save a code cell's content using the Jupiter magic `%% writefile filename`


In [8]:
%%writefile going_modular/going_modular/data_setup.py
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

def create_dataloaders(
        train_dir: str,
        test_dir: str,
        transform: transforms.Compose,
        batch_size: int,
        num_workers: int
):
    '''
    Creates train and test dataloaders

    takes in a training directory and testing directory path and turns them into
    Pytorch Datasets and then into PyTorch Dataloaders.

    Args:
        train_dir: Path to training directory
        test_dir: Path to testing directory
        transform: torchvision transforms to perform on training and testing data
        batch_size: size of each image batch
        num_workers: number of subprocesses to use for data loading

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
        Either 'pizza', 'steak', 'sushi'
    '''
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)
    print(f"Train data:\n{train_data}\nTest data:\n{test_data}")
    # Get class names as a list
    class_names = train_data.classes
    # Turn train and test Datasets into DataLoaders
    train_dataloader = DataLoader(dataset=train_data, 
                                batch_size=batch_size, # how many samples per batch?
                                num_workers=num_workers, # how many subprocesses to use for data loading? (higher = more)
                                shuffle=True,
                                pin_memory=True) # put data in pinned memory for faster transfer

    test_dataloader = DataLoader(dataset=test_data, 
                                batch_size=batch_size, 
                                num_workers=num_workers, 
                                shuffle=False,
                                pin_memory=True) 
    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/going_modular/data_setup.py


## 3. Making a model (TinyVGG)

We're going to use the same model we used in notebook 04: TinyVGG from the CNN Explainer website.

The only change here from notebook 04 is that a docstring has been added using [Google's Style Guide for Python](https://google.github.io/styleguide/pyguide.html#384-classes). 

In [9]:
%%writefile going_modular/going_modular/model_builder.py
import torch
from torch import nn

class TinyVGG(nn.Module):
    """
    Model architecture copying TinyVGG from: 
    https://poloclub.github.io/cnn-explainer/

    Args:
        num_blocks: Number of convolutional layers
        input_shape: Number of channels in the input
        hidden_units: Number of hidden units
        output_shape: Number of channels in the output
    """
    def __init__(self, num_blocks: int, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        conv_blocks = []
        out_conv_blocks = 64
        for _ in range(num_blocks):
            conv_blocks.append(
                nn.Sequential(
                    nn.Conv2d(in_channels=input_shape,
                              out_channels=hidden_units,
                              kernel_size=3,
                              stride=1,
                              padding=1),
                    nn.ReLU(),
                    nn.Conv2d(in_channels=hidden_units,
                              out_channels=hidden_units,
                              kernel_size=3,
                              stride=1,
                              padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=2, stride=2)
                )
            )
            input_shape = hidden_units
            out_conv_blocks = out_conv_blocks // 2
        # Transform list of conv_blocks into a sequence of layers
        self.conv_blocks = nn.Sequential(*conv_blocks)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # It's because each layer of our network compresses and changes the shape of our input data.
            nn.Linear(in_features=hidden_units*out_conv_blocks*out_conv_blocks, # we divide by 2 for each conv_blocks
                      out_features=output_shape)
        )
    def forward(self, x: torch.Tensor):
        return self.classifier(self.conv_blocks(x)) # <- leverage the benefits of operator fusion
        

Overwriting going_modular/going_modular/model_builder.py


## 4. Creating `train_test_step` function

In [10]:
%%writefile going_modular/going_modular/engine.py
from typing import Dict, List
from timeit import default_timer as timer
import torch
from tqdm.auto import tqdm

def train_test_step(model: torch.nn.Module, 
                    train_dataloader: torch.utils.data.DataLoader, 
                    test_dataloader: torch.utils.data.DataLoader, 
                    loss_fn: torch.nn.Module, 
                    optimizer: torch.optim.Optimizer, 
                    device: torch.device,
                    epochs: int = 5) -> Dict[str, List[float]]:
    """
    Trains and tests a PyTorch model

    Turns a target PyTorch model to training mode and then
    runs the forward and backward passes on a training dataset.
    It also calculates and returns the loss and accuracy on the test dataset

    Args:
      model: A PyTorch model to be trained and tested
      train_dataloader: A DataLoader instance for the model to be trained on
      test_dataloader: A DataLoader instance for the model to be tested on
      loss_fn: A PyTorch loss function to minimize
      optimizer: A PyTorch optimizer to help minimize the loss function
      device: A target device to compute on (e.g. "cuda" or "cpu")

    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for 
      each epoch.
    """
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    torch.manual_seed(42)
    train_start = timer()
    for epoch in tqdm(range(epochs)):
        print(f"Epoch {epoch}\n-------")
        train_loss = 0
        train_acc = 0
        for batch, (X,y) in enumerate(train_dataloader):
            X, y = X.to(device), y.to(device)
            model.train().to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            train_loss += loss
            train_acc += ((torch.eq(torch.argmax(dim=1, input=y_pred), y)).sum().item()/len(y_pred))*100
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if batch % 400 == 0:
                print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")
        train_loss /= len(train_dataloader)
        train_acc /= len(train_dataloader)
        test_loss = 0
        test_acc = 0
        model.eval().to(device)
        with torch.inference_mode():
            for X, y in test_dataloader:
                X, y = X.to(device), y.to(device)
                test_pred = model(X)
                test_loss += loss_fn(test_pred, y)
                test_acc += ((torch.eq(torch.argmax(dim=1, input=test_pred), y)).sum().item()/len(test_pred))*100
            test_loss /= len(test_dataloader)
            test_acc /= len(test_dataloader)
        results["train_loss"].append(train_loss.item())
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss.item())
        results["test_acc"].append(test_acc)
        print(f"Train loss: {train_loss:.3f} | Train accuracy: {train_acc:.2f}%")
        print(f"Test loss: {test_loss:.3f} | Test accuracy: {test_acc:.2f}%")
    train_end = timer()
    total_time = train_end - train_start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return results


Overwriting going_modular/going_modular/engine.py


## 5. Creating a function to save the model (script mode)

How about we add our `save_model()` function to a script called `utils.py` which is short for "utilities".

We can do so with the magic line `%%writefile going_modular/utils.py`.

In [11]:
%%writefile going_modular/going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
from pathlib import Path
from typing import List, Dict
import matplotlib.pyplot as plt

import torch

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)


def load_model(model_path: str,
               model_builder: torch.nn.Module,
               device: torch.device,
               input_shape: int,
               output_shape: int,
               hidden_units: int,
               num_blocks: int):
    
    '''
    Loads a PyTorch model from a target directory.

    Args:
    model_path: A directory for saving the model to.
    model_builder: A model builder to use to create the model.
    device: A target device to compute on (e.g. "cuda" or "cpu").
    input_shape: Number of channels in the input
    output_shape: Number of channels in the output
    hidden_units: Number of hidden units
    num_blocks: Number of convolutional layers

    Returns:
    A PyTorch model from a target directory
    '''
    loaded_model = model_builder(input_shape=input_shape,
                                 hidden_units=hidden_units,
                                 output_shape=output_shape,
                                 num_blocks=num_blocks
                                ).to(device)
    loaded_model.load_state_dict(torch.load(model_path))
    return loaded_model


def get_classes(image_dir):

  train_path = Path(image_dir)
  class_names = [d.name for d in train_path.iterdir() if d.is_dir()]
  return class_names


def plot_loss_curves(results: Dict[str, List[float]]):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """
    
    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot 
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

Overwriting going_modular/going_modular/utils.py


## 6. Train, evaluate and save the model

Let's leverage the functions we've got above to train, test and save a model to file.

In [12]:
%%writefile going_modular/going_modular/train.py
import os
import torch
from torch import nn
from torchvision import transforms
from data_import import get_data
from data_setup import create_dataloaders
from engine import train_test_step
from model_builder import TinyVGG
from utils import save_model


if __name__ == "__main__":

   # Setup hyperparameters
  NUM_EPOCHS = 5
  NUM_BLOCKS = 2
  INPUT_SHAPE = 3
  BATCH_SIZE = 32
  HIDDEN_UNITS = 10
  LEARNING_RATE = 0.001
  NUM_WORKERS = os.cpu_count()


  # Setup target device
  device = "cuda" if torch.cuda.is_available() else "cpu"

  # Get data
  get_data(url="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
          file="pizza_steak_sushi")

  # Setup directories
  train_dir = "data/pizza_steak_sushi/train"
  test_dir = "data/pizza_steak_sushi/test"


  # Create transforms
  data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
  ])

  # Create DataLoaders with help from data_setup.py
  train_dataloader, test_dataloader, class_names = create_dataloaders(
      train_dir=train_dir,
      test_dir=test_dir,
      transform=data_transform,
      batch_size=BATCH_SIZE,
      num_workers=NUM_WORKERS
  )

  # Create model with help from model_builder.py
  model = TinyVGG(
      input_shape=INPUT_SHAPE,
      hidden_units=HIDDEN_UNITS,
      output_shape=len(class_names),
      num_blocks=NUM_BLOCKS
  ).to(device)

  # Set loss and optimizer
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),
                              lr=LEARNING_RATE)

  # Start training with help from engine.py
  results = train_test_step(model=model,
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            epochs=NUM_EPOCHS,
            device=device)

  # Save the model with help from utils.py
  save_model(model=model,
            target_dir="models",
            model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting going_modular/going_modular/train.py


In [22]:
%%writefile going_modular/going_modular/predict.py
from PIL import Image
import matplotlib.pyplot as plt
import torch

def predict_image(img_path, model, class_names, device, image_transform):
    '''
    Predict the class of a single image.

    Args:
    img_path: A string path to an image file.
    model: A trained PyTorch model.
    class_names: A list of class names.
    device: A target device to compute on (e.g. "cuda" or "cpu").
    image_transform: A transform to apply to the image.
    '''

    img = Image.open(img_path)
    img_transformed = image_transform(img).to(device)
    model.eval()
    with torch.inference_mode():
        pred_logit = model(img_transformed.unsqueeze(0).to(device))
        pred_probs = torch.softmax(pred_logit, dim=1)[0]
        pred_classes = pred_logit.argmax(1)
    fig = plt.figure(figsize=(5, 5))
    plt.imshow(img)
    plt.title(f"Prediction: {class_names[pred_classes]} | Probability: {pred_probs[pred_classes].item()*100:.2f} %")
    print(f"Prediction: {class_names[pred_classes]} | Probability: {pred_probs[pred_classes].item()*100:.2f} %")
    plt.axis("off")    

Overwriting going_modular/going_modular/predict.py


In [14]:
%%writefile going_modular/going_modular/test.py
from torchvision import transforms
from utils import load_model, get_classes
from model_builder import TinyVGG
from predict import predict_image
import torch

HIDDEN_UNITS = 10
NUM_BLOCKS = 2
INPUT_SHAPE = 3

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

# get classes
class_names = get_classes(image_dir="data/pizza_steak_sushi/train")

# Setup transforms
image_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Load model
model = load_model(model_path="models/05_going_modular_script_mode_tinyvgg_model.pth",
                   model_builder=TinyVGG,
                   device=device,
                   input_shape=INPUT_SHAPE,
                   output_shape=len(class_names),
                   hidden_units=HIDDEN_UNITS,
                   num_blocks=NUM_BLOCKS)

# Predict
predict_image(img_path="images/istockphoto-540233806-612x612.jpg",
              model=model,
              class_names=class_names,
              device=device,
              image_transform=image_transform)

Overwriting going_modular/going_modular/test.py
